In [7]:
import os
import numpy as np
import pandas as pd

os.chdir(r"/Users/sanghyuk/Documents/preprocessing_python/lecture_source/2. 탐색적 데이터 분석/데이터/")

In [9]:
df = pd.read_csv("E-Commerce_UK.csv", encoding='CP949')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,C-17850
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,C-17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,C-17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,C-17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,C-17850


#### 데이터 분할

In [11]:
refund_df = df.loc[df['Quantity'] <= 0] # 환불 데이터
order_df = df.loc[df['Quantity'] > 0] # 주문 데이터

#### 고객별 특징 추출

In [12]:
# 클러스터링을 위한 데이터 초기화
cluster_df = pd.DataFrame({"CustomerID":order_df['CustomerID'].unique()})
cluster_df

,CustomerID
0,C-17850
1,C-13047
2,C-12583
3,C-13748
4,C-15100
...,...
4339,C-13436
4340,C-15520
4341,C-13298
4342,C-14569


하단 drop_duplicate 코드는, 한 주문에서 여러 물건 상 경우를 제거해 주는 것. 

In [21]:
number_of_order_per_CID.to_dict().values()

dict_values([224, 217, 132, 107, 98, 96, 89, 75, 68, 66, 64, 63, 61, 60, 57, 55, 49, 48, 48, 47, 47, 46, 45, 44, 41, 41, 41, 40, 40, 39, 38, 38, 38, 37, 36, 36, 35, 35, 34, 34, 34, 34, 33, 33, 33, 32, 32, 32, 32, 31, 30, 30, 30, 30, 29, 29, 29, 29, 29, 29, 28, 28, 28, 27, 27, 27, 27, 27, 27, 26, 26, 26, 26, 26, 26, 26, 26, 25, 25, 25, 25, 24, 24, 24, 24, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 22, 22, 22, 22, 22, 22, 22, 21, 21, 21, 21, 21, 21, 21, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 13,

In [13]:
# 주문 횟수 계산 및 부착
number_of_order_per_CID = order_df.drop_duplicates(subset = ['CustomerID', 'InvoiceNo'])['CustomerID'].value_counts()
cluster_df['주문횟수'] = cluster_df['CustomerID'].replace(number_of_order_per_CID.to_dict())

# 주문 횟수가 0인 경우에는 replace가 되지 않아 CustomerID가 부착될 수 있음
# 따라서 이러한 경우를 대비하기 위해 0으로 변경
cluster_df.loc[cluster_df['CustomerID'] == cluster_df['주문횟수'], '주문횟수'] = 0
cluster_df.head()

,CustomerID,주문횟수
0,C-17850,34
1,C-13047,21
2,C-12583,15
3,C-13748,5
4,C-15100,4


In [23]:
(cluster_df['CustomerID'] == cluster_df['주문횟수']).value_counts()

False    4344
dtype: int64

In [26]:
# 반품 횟수 계산 및 부착 
number_of_refund_per_CID = refund_df.drop_duplicates(subset = ['CustomerID', 'InvoiceNo'])['CustomerID'].value_counts()
cluster_df['반품횟수'] = cluster_df['CustomerID'].replace(number_of_refund_per_CID.to_dict())

cluster_df.loc[cluster_df['CustomerID'] == cluster_df['반품횟수'], '반품횟수'] = 0
cluster_df.head()

,CustomerID,주문횟수,반품횟수
0,C-17850,34,1
1,C-13047,21,8
2,C-12583,15,3
3,C-13748,5,0
4,C-15100,4,3


In [27]:
# 주문량 계산 및 부착
number_of_quantity_per_CID = order_df.groupby('CustomerID')['Quantity'].sum()
cluster_df['주문량'] = cluster_df['CustomerID'].replace(number_of_quantity_per_CID.to_dict())
cluster_df.loc[cluster_df['CustomerID'] == cluster_df['주문량'], '주문량'] = 0
cluster_df.head()

,CustomerID,주문횟수,반품횟수,주문량
0,C-17850,34,1,1733
1,C-13047,21,8,1953
2,C-12583,15,3,5060
3,C-13748,5,0,439
4,C-15100,4,3,81


In [28]:
# 주문금액합계 계산 및 부착
order_df.loc[:, '주문금액'] = order_df.loc[:, 'Quantity'] * order_df.loc[:, 'UnitPrice']

number_of_quantity_per_CID = order_df.groupby('CustomerID')['주문금액'].sum()
cluster_df['주문금액합계'] = cluster_df['CustomerID'].replace(number_of_quantity_per_CID.to_dict())
cluster_df.loc[cluster_df['CustomerID'] == cluster_df['주문금액합계'], '주문금액합계'] = 0
cluster_df.head()

/Users/sanghyuk/anaconda/envs/py38/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/sanghyuk/anaconda/envs/py38/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,CustomerID,주문횟수,반품횟수,주문량,주문금액합계
0,C-17850,34,1,1733,5391.21
1,C-13047,21,8,1953,6619.51
2,C-12583,15,3,5060,7281.38
3,C-13748,5,0,439,948.25
4,C-15100,4,3,81,877.25


In [30]:
order_df.drop_duplicates(subset = ['CustomerID', 'InvoiceDate'], keep = 'last')[['CustomerID', 'InvoiceDate']]

,CustomerID,InvoiceDate
6,C-17850,12/1/2010 8:26
8,C-17850,12/1/2010 8:28
24,C-13047,12/1/2010 8:34
25,C-13047,12/1/2010 8:35
45,C-12583,12/1/2010 8:45
...,...,...
539349,C-13777,12/9/2011 12:23
539351,C-13777,12/9/2011 12:25
539372,C-15804,12/9/2011 12:31
539376,C-13113,12/9/2011 12:49


In [32]:
# 최근 주문 ~ 현재 시점까지 거리 부착
current_date = pd.to_datetime('2011-12-10') # datetime: 날짜 자료형
def recency(value):   # 하나의 element를 처리함! 2010-10-01
    month, day, year = value.split(' ')[0].split('/')    
    diff = (current_date - pd.to_datetime('{}-{}-{}'.format(year, month, day))).days
    return diff

# keep = last를 통해, 계산량 감소
order_df_without_duplicates = order_df.drop_duplicates(subset = ['CustomerID', 'InvoiceDate'], keep = 'last')[['CustomerID', 'InvoiceDate']]
order_df_without_duplicates['최근성'] = order_df_without_duplicates['InvoiceDate'].apply(recency)

min_recency_per_CID = order_df_without_duplicates.set_index('CustomerID')['최근성']
cluster_df['최근성'] = cluster_df['CustomerID'].replace(min_recency_per_CID.to_dict())

In [33]:
min_recency_per_CID

CustomerID
C-17850    374
C-17850    374
C-13047    374
C-13047    374
C-12583    374
          ... 
C-13777      1
C-13777      1
C-15804      1
C-13113      1
C-12680      1
Name: 최근성, Length: 19697, dtype: int64

In [34]:
cluster_df.head()

,CustomerID,주문횟수,반품횟수,주문량,주문금액합계,최근성
0,C-17850,34,1,1733,5391.21,373
1,C-13047,21,8,1953,6619.51,32
2,C-12583,15,3,5060,7281.38,3
3,C-13748,5,0,439,948.25,96
4,C-15100,4,3,81,877.25,331


In [35]:
cluster_df.set_index('CustomerID', inplace = True)

#### 고객의 주문 특성에 따른 군집화 수행

In [36]:
# 군집화 모델 인스턴스화 및 학습
from sklearn.cluster import AgglomerativeClustering as AC
clustering_model = AC(n_clusters = 5,
                      affinity = "cosine",
                      linkage = "average")
clustering_model.fit(cluster_df)

AgglomerativeClustering(affinity='cosine', linkage='average', n_clusters=5)

In [39]:
cluster_df['주문특성_군집'] = clustering_model.labels_
cluster_df.head()

,주문횟수,반품횟수,주문량,주문금액합계,최근성,주문특성_군집
CustomerID,,,,,,
C-17850,34,1,1733,5391.21,373,0
C-13047,21,8,1953,6619.51,32,0
C-12583,15,3,5060,7281.38,3,0
C-13748,5,0,439,948.25,96,0
C-15100,4,3,81,877.25,331,0


In [40]:
cluster_df.groupby(['주문특성_군집'])['주문횟수', '반품횟수', '주문량', '주문금액합계', '최근성'].mean()

<ipython-input-40-e6b5e7f7a7dc>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  cluster_df.groupby(['주문특성_군집'])['주문횟수', '반품횟수', '주문량', '주문금액합계', '최근성'].mean()


,주문횟수,주문량,주문금액합계,최근성
주문특성_군집,,,,
0,5.482738,1502.736633,3073.161575,59.893064
1,3.370717,1823.274143,1395.034455,69.040498
2,1.345733,148.695842,272.678600,231.516411
3,1.086331,59.593525,127.332122,272.694245
4,1.066667,8.466667,27.213333,293.200000


#### 주요 상품 확인

In [41]:
order_df['Description'].value_counts().iloc[:100]

WHITE HANGING HEART T-LIGHT HOLDER    2323
JUMBO BAG RED RETROSPOT               2112
REGENCY CAKESTAND 3 TIER              2017
PARTY BUNTING                         1706
LUNCH BAG RED RETROSPOT               1594
                                      ... 
LUNCH BAG DOLLY GIRL DESIGN            698
SET OF 3 HEART COOKIE CUTTERS          696
ANTIQUE SILVER T-LIGHT GLASS           696
JUMBO BAG SPACEBOY DESIGN              696
GINGERBREAD MAN COOKIE CUTTER          689
Name: Description, Length: 100, dtype: int64

In [43]:
from mlxtend.preprocessing import TransactionEncoder
encoder = TransactionEncoder()

product_list_per_customer = order_df.groupby(['CustomerID'])['StockCode'].apply(list)

one_hot_df = encoder.fit(product_list_per_customer).transform(product_list_per_customer) # 결과: ndarray
one_hot_df = pd.DataFrame(one_hot_df, columns = encoder.columns_, index = product_list_per_customer.index)
one_hot_df.head() # 매우 희소

,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,M,PADS,POST,S,gift_0001_10,gift_0001_20,gift_0001_30,gift_0001_40,gift_0001_50,m
CustomerID,,,,,,,,,,,,,,,,,,,,,
C-12346,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
C-12347,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
C-12348,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
C-12349,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
C-12350,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [44]:
product_list_per_customer

CustomerID
C-12346                                              [23166]
C-12347    [85116, 22375, 71477, 22492, 22771, 22772, 227...
C-12348    [84992, 22951, 84991, 84991, 21213, 21213, 226...
C-12349    [23112, 23460, 21564, 21411, 21563, 22131, 221...
C-12350    [21908, 22412, 79066K, 79191C, 22348, 84086C, ...
                                 ...                        
C-18280    [82484, 22180, 22467, 22725, 22727, 22495, 223...
C-18281    [22037, 22716, 22028, 23007, 23008, 23209, 22467]
C-18282    [21270, 23187, 23295, 22089, 21108, 21109, 224...
C-18283    [22356, 20726, 22384, 22386, 20717, 20718, 850...
C-18287    [22755, 22754, 22753, 22756, 22758, 22757, 227...
Name: StockCode, Length: 4344, dtype: object

In [54]:
# 최소 100회 이상 나온 품목만 사용
one_hot_df = one_hot_df.loc[:, one_hot_df.columns[one_hot_df.sum(axis = 0) > 100]]


In [45]:
clustering_model = AC(n_clusters = 5,
                      affinity = "jaccard",
                      linkage = "average")
clustering_model.fit(one_hot_df)

AgglomerativeClustering(affinity='jaccard', linkage='average', n_clusters=5)

In [46]:
labels = clustering_model.labels_
one_hot_df['군집'] = labels

In [47]:
pd.Series(labels).value_counts()

0    4340
3       1
2       1
1       1
4       1
dtype: int64